In [1]:
## Instalações

# Acessar os valores das ações, petróleo (brent), ibovespa e minério de ferro
# !pip install yfinance

# Acessar a variável de entrada do fechamento do dólar
# !pip install python-bcb==0.3.3

In [2]:
## Bibliotecas

# Pre Processamento dos dados
import pandas as pd
import numpy as np

# Download das Ações
import yfinance as yf
from bcb import sgs

# Montagem dos Dados
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Baixar os Dados já montados
import joblib
import os

In [3]:
## Definição de Variáveis fixas

acao = ''

start_treinamento = '2016-09-01'
end_treinamento = '2023-09-01'

start_validacao = '2023-09-02'
end_validacao = '2025-09-02'

ticker = f'{acao}.SA'

In [4]:
## Definição das Features fixas PARA TESTE

# Dólar
dolar_treinamento = sgs.get({'Close_Dolar': 1}, start=start_treinamento, end=end_treinamento).reset_index() # Faz com que Date deixe de ser index e passe a ser uma coluna junto ao Close_Dolar
dolar_treinamento['Date'] = pd.to_datetime(dolar_treinamento['Date']) # Muda o type da coluna Date para Datetime para ficar igual as demais variáveis
dolar_treinamento.set_index('Date', inplace=True) # Retorna Date para index, pois é assim que as demais variáveis estão

# Petróleo (WTI) pelo Yahoo Finance
petroleo_treinamento = yf.download('CL=F', start=start_treinamento, end=end_treinamento)[['Close']] # Preço do barril de petróleo WTI
petroleo_treinamento.columns = ['Close_Petroleo']

# Ibovespa
ibov_treinamento = yf.download('^BVSP', start=start_treinamento, end=end_treinamento)[['Close']] # Índice ibovespa
ibov_treinamento.columns = ['Close_Ibovespa']

# Minério de Ferro (supondo índice internacional)
minerio_treinamento = yf.download('RIO', start=start_treinamento, end=end_treinamento)[['Close']] # Ação da mineradora Rio Tinto
minerio_treinamento.columns = ['Close_Minerio']

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\4201631905.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  petroleo_treinamento = yf.download('CL=F', start=start_treinamento, end=end_treinamento)[['Close']] # Preço do barril de petróleo WTI
[*********************100%***********************]  1 of 1 completed
C:\Users\felip\AppData\Local\Temp\ipykernel_9232\4201631905.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ibov_treinamento = yf.download('^BVSP', start=start_treinamento, end=end_treinamento)[['Close']] # Índice ibovespa
[*********************100%***********************]  1 of 1 completed
C:\Users\felip\AppData\Local\Temp\ipykernel_9232\4201631905.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  minerio_treinamento = yf.download('RIO', start=start_treinamento, end=end_treinamento)[['Close']] # Ação da mineradora Rio Tinto
[*********************100%******************

In [5]:
## Definição das Features fixas PARA VALIDAÇÃO

# Dólar
dolar_validacao = sgs.get({'Close_Dolar': 1}, start=start_validacao, end=end_validacao).reset_index() # Faz com que Date deixe de ser index e passe a ser uma coluna junto ao Close_Dolar
dolar_validacao['Date'] = pd.to_datetime(dolar_validacao['Date']) # Muda o type da coluna Date para Datetime para ficar igual as demais variáveis
dolar_validacao.set_index('Date', inplace=True) # Retorna Date para index, pois é assim que as demais variáveis estão

# Petróleo (WTI) pelo Yahoo Finance
petroleo_validacao = yf.download('CL=F', start=start_validacao, end=end_validacao)[['Close']] # Preço do barril de petróleo WTI
petroleo_validacao.columns = ['Close_Petroleo']

# Ibovespa
ibov_validacao = yf.download('^BVSP', start=start_validacao, end=end_validacao)[['Close']] # Índice ibovespa
ibov_validacao.columns = ['Close_Ibovespa']

# Minério de Ferro (supondo índice internacional)
minerio_validacao = yf.download('RIO', start=start_validacao, end=end_validacao)[['Close']] # Ação da mineradora Rio Tinto
minerio_validacao.columns = ['Close_Minerio']

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\627460923.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  petroleo_validacao = yf.download('CL=F', start=start_validacao, end=end_validacao)[['Close']] # Preço do barril de petróleo WTI
[*********************100%***********************]  1 of 1 completed
C:\Users\felip\AppData\Local\Temp\ipykernel_9232\627460923.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ibov_validacao = yf.download('^BVSP', start=start_validacao, end=end_validacao)[['Close']] # Índice ibovespa
[*********************100%***********************]  1 of 1 completed
C:\Users\felip\AppData\Local\Temp\ipykernel_9232\627460923.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  minerio_validacao = yf.download('RIO', start=start_validacao, end=end_validacao)[['Close']] # Ação da mineradora Rio Tinto
[*********************100%***********************]  1 of 1 comple

In [6]:
## Funções

def baixar_dados_acao(acao, start, end):
    df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
    df.columns = [f'Close_{acao}', f'Volume_{acao}']
    return df

def preparar_dataframe_completo(df_acao, dolar, petroleo, ibov, minerio):
    df = df_acao.copy()
    df = df.merge(dolar, left_index=True, right_index=True, how='left')
    df = df.merge(petroleo, left_index=True, right_index=True, how='left')
    df = df.merge(ibov, left_index=True, right_index=True, how='left')
    df = df.merge(minerio, left_index=True, right_index=True, how='left')
    df.dropna(inplace=True)
    return df

def normalizar_e_salvar(df, acao, pasta, split_ratio=0.8):
    n_total = len(df)
    split = int(n_total * split_ratio)
    scaler = MinMaxScaler()
    scaler.fit(df.iloc[:split].values.astype(np.float64))
    dados_escalados = scaler.transform(df.values.astype(np.float64))
    joblib.dump(scaler, f'./{pasta}/{acao}/scaler_{acao}.pkl')
    return dados_escalados, scaler

def createXY(dataset, n_past):
    dataX, dataY = [], []
    for i in range(n_past, len(dataset)):
        dataX.append(dataset[i - n_past:i, :])
        dataY.append(dataset[i, 0])
    return np.array(dataX), np.array(dataY)

def gerar_janelas_e_salvar(dados_escalados, df, acao, pasta, janela=21):
    X, y = createXY(dados_escalados, janela)
    y_real = df.iloc[janela:][f'Close_{acao}'].values.reshape(-1, 1)

    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    y_real_train, y_real_test = y_real[:split], y_real[split:]

    df.to_csv(f'./Excel/{pasta}/df_completo_{acao}.csv', index_label="Date")

    np.save(f'./{pasta}/{acao}/X_train_{acao}.npy', X_train)
    np.save(f'./{pasta}/{acao}/y_train_{acao}.npy', y_train)
    np.save(f'./{pasta}/{acao}/X_test_{acao}.npy', X_test)
    np.save(f'./{pasta}/{acao}/y_test_{acao}.npy', y_test)
    np.save(f'./{pasta}/{acao}/y_real_train_{acao}.npy', y_real_train)
    np.save(f'./{pasta}/{acao}/y_real_{acao}.npy', y_real_test)

In [7]:
os.makedirs("Treinamento", exist_ok=True)
os.makedirs("Validação", exist_ok=True)

os.makedirs("Excel", exist_ok=True)
os.makedirs(f'Excel/Treinamento', exist_ok=True)
os.makedirs(f'Excel/Validação', exist_ok=True)

## AÇÃO DA ABEV3


In [8]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'ABEV3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [9]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'ABEV3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA B3SA3

In [10]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'B3SA3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [11]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'B3SA3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA BBAS3

In [12]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'BBAS3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [13]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'BBAS3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA BBDC4

In [14]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'BBDC4'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [15]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'BBDC4'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA ELET3

In [16]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'ELET3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [17]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'ELET3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA ITUB4

In [18]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'ITUB4'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [19]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'ITUB4'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA PETR3

In [20]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'PETR3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [21]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'PETR3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA PETR4

In [22]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'PETR4'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [23]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'PETR4'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA SBSP3

In [24]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'SBSP3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [25]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'SBSP3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


## AÇÃO DA VALE3

In [26]:
# DOWNLOAD DOS DADOS DE TREINAMENTO

acao = 'VALE3'
pasta = 'Treinamento'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_treinamento, end_treinamento)
df = preparar_dataframe_completo(dados, dolar_treinamento, petroleo_treinamento, ibov_treinamento, minerio_treinamento)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed


In [27]:
# DOWNLOAD DOS DADOS DE VALIDAÇÃO

acao = 'VALE3'
pasta = 'Validação'

os.makedirs(f'{pasta}/{acao}', exist_ok=True)

dados = baixar_dados_acao(acao, start_validacao, end_validacao)
df = preparar_dataframe_completo(dados, dolar_validacao, petroleo_validacao, ibov_validacao, minerio_validacao)
dados_escalados, scaler = normalizar_e_salvar(df, acao, pasta)
gerar_janelas_e_salvar(dados_escalados, df, acao, pasta)

C:\Users\felip\AppData\Local\Temp\ipykernel_9232\1810153427.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(f'{acao}.SA', start=start, end=end)[['Close', 'Volume']]
[*********************100%***********************]  1 of 1 completed
